# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [3]:
path = 'Datasets/'
playstore = spark.read.csv(path+"googleplaystore.csv",header=True,inferSchema=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [4]:
playstore.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [7]:
from pyspark.sql.types import IntegerType, FloatType
newdf = playstore.withColumn("Rating", playstore["Rating"].cast(FloatType())) \
            .withColumn("Reviews", playstore["Reviews"].cast(IntegerType())) \
            .withColumn("Price", playstore["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [11]:
newdf.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [15]:
spark.sql("SELECT * FROM tempview WHERE rating > 4.1 limit 10").toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
5,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up
6,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,28M,"1,000,000+",Free,0,Everyone,Art & Design,"October 27, 2017",1.0.4,4.1 and up
7,Name Art Photo Editor - Focus n Filters,ART_AND_DESIGN,4.4,8788,12M,"1,000,000+",Free,0,Everyone,Art & Design,"July 31, 2018",1.0.15,4.0 and up
8,Tattoo Name On My Photo Editor,ART_AND_DESIGN,4.2,44829,20M,"10,000,000+",Free,0,Teen,Art & Design,"April 2, 2018",3.8,4.1 and up
9,Mandala Coloring Book,ART_AND_DESIGN,4.6,4326,21M,"100,000+",Free,0,Everyone,Art & Design,"June 26, 2018",1.0.4,4.4 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [37]:
selected_data = spark.sql("SELECT App,Rating, category FROM tempview WHERE rating > 4.5 and lower(category)='comics' ")
selected_data.toPandas()


,App,Rating,category
0,Manga Master - Best manga & comic reader,4.6,COMICS
1,GANMA! - All original stories free of charge f...,4.7,COMICS
2,Röhrich Werner Soundboard,4.7,COMICS
3,Unicorn Pokez - Color By Number,4.8,COMICS
4,Manga - read Thai translation,4.6,COMICS
5,Dragon Ball Wallpaper - Ringtones,4.7,COMICS
6,Children's cartoons (Mithu-Mina-Raju),4.6,COMICS
7,【Ranobbe complete free】 Novelba - Free app tha...,NaN,COMICS
8,Faustop Sounds,4.7,COMICS
9,Hojiboy Tojiboyev Life Hacks,5.0,COMICS


In [35]:
spark.sql("SELECT distinct(category) FROM tempview").show()

+-------------------+
|           category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|               Face|
+-------------------+
only showing top 20 rows



## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [41]:
spark.sql("SELECT Category, sum(reviews) AS Total_Reviews FROM tempview GROUP BY Category order by Total_Reviews DESC ").limit(1).toPandas()

,Category,Total_Reviews
0,GAME,1585422349


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [45]:
spark.sql("SELECT App, reviews FROM tempview GROUP BY App,reviews order by sum(Reviews) DESC ").limit(1).toPandas()

,App,reviews
0,WhatsApp Messenger,69119316


## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [48]:
spark.sql("SELECT App, reviews FROM tempview WHERE lower(App) like '%dating%' ").limit(10).toPandas()

,App,reviews
0,"Private Dating, Hide App- Blue for PrivacyHider",0
1,AMBW Dating App: Asian Men Black Women Interra...,2
2,Zoosk Dating App: Meet Singles,516801
3,OkCupid Dating,285726
4,Match™ Dating - Meet Singles,76646
5,"Hily: Dating, Chat, Match, Meet & Hook up",2556
6,Hinge: Dating & Relationships,7779
7,Casual Dating & Adult Singles - Joyride,61637
8,BBW Dating & Plus Size Chat,12632
9,CMB Free Dating App,48845


## 6. Use SQL Transformer to display how many free apps there are in this list

In [50]:
from pyspark.ml.feature import SQLTransformer

In [55]:
sqlTrans = SQLTransformer(
    statement="SELECT count(app) FROM __THIS__ WHERE lower(Type)='free' ")
sqlTrans.transform(newdf).toPandas()

,count(app)
0,10037


## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [59]:
sqlTrans = SQLTransformer(
    statement="SELECT category,count(category) as total FROM __THIS__ group by category order by total desc limit 1")
sqlTrans.transform(newdf).toPandas()

,category,total
0,FAMILY,1972


## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [62]:
spark.sql("SELECT * FROM tempview WHERE lower(category) = 'tools' and reviews >100").limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Moto File Manager,TOOLS,4.1,38655,5.9M,"10,000,000+",Free,0,Everyone,Tools,"February 1, 2018",v3.7.93,5.0 and up
1,Google,TOOLS,4.4,8033493,Varies with device,"1,000,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,Varies with device
2,Google Translate,TOOLS,4.4,5745093,Varies with device,"500,000,000+",Free,0,Everyone,Tools,"August 4, 2018",Varies with device,Varies with device
3,Moto Display,TOOLS,4.2,18239,Varies with device,"10,000,000+",Free,0,Everyone,Tools,"August 6, 2018",Varies with device,Varies with device
4,Motorola Alert,TOOLS,4.2,24199,3.9M,"50,000,000+",Free,0,Everyone,Tools,"November 21, 2014",1.02.53,4.4 and up
5,Motorola Assist,TOOLS,4.1,37333,Varies with device,"50,000,000+",Free,0,Everyone,Tools,"January 17, 2016",Varies with device,Varies with device
6,Cache Cleaner-DU Speed Booster (booster & clea...,TOOLS,4.5,12759663,15M,"100,000,000+",Free,0,Everyone,Tools,"July 25, 2018",3.1.2,4.0 and up
7,Moto Suggestions ™,TOOLS,4.6,308,4.3M,"1,000,000+",Free,0,Everyone,Tools,"June 8, 2018",0.2.32,8.0 and up
8,Moto Voice,TOOLS,4.1,33216,Varies with device,"10,000,000+",Free,0,Everyone,Tools,"June 5, 2018",Varies with device,Varies with device
9,Calculator,TOOLS,4.3,40770,Varies with device,"100,000,000+",Free,0,Everyone,Tools,"November 21, 2017",Varies with device,Varies with device


## That's all folks! Great job!